### generate mock point source data
#### main function ：F(x) `generate_mockCMD`
- input x : 
  - [Fe/H], Age, (DM), $\rm{f}_{b}$, IMF($\alpha$)
  - m(Error), mag range, $\rm{N}_{partical}$

- output y :
  - G, BP-RP ==> CMD

#### functions inside F(x) :
1. isochrone([Fe/H], Age, (DM))
  
2. Sample mockstar(IMF, $\rm{f}_{b}$)
 
3. add Error 

In [6]:
from berliner import CMD
from scipy import integrate
import scipy.interpolate as spl
from joblib import Parallel,delayed
import random
import re
import glob
import pandas as pd
import numpy as np
import emcee

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("default")

import sys
sys.path.append('../') # 添加包含functionkit.py文件的目录路径
import functionkit

#### 1. isochrone([Fe/H], Age, (DM))

- get isochrones using berliner 

In [7]:
def get_isochrones(grid_logage,grid_mh):
    '''
    get isochrones using berliner
    define your grid of logAge and [M/H] as tuple(lower, upper, step)
    grid_logage = (6, 10.2, 0.1)
    grid_mh = (-2.6, 0.5, 0.1)
    '''
    c = CMD()
    # download isochrones in parallel
    isoc_lgage, isoc_mhini, isoc_list = c.get_isochrone_grid_mh(
        grid_logage=grid_logage, grid_mh=grid_mh, photsys_file="gaiaDR2",
        n_jobs=50, verbose=10)

    # save isochrone to csv file
    for i in range(len(isoc_list)):
        iso = isoc_list[i].to_pandas()
        #iso = iso[(iso['label']>0)&(iso['label']<=7)]
        logAge = iso.iloc[0]['logAge']
        MH = iso.iloc[0]['MH']
        iso.to_csv('../../data/isochrones/iso_age_%s_mh_%s.csv'%(logAge,MH),index=False)
        
grid_logage = (6, 10.2, 0.1)
grid_mh = (-2.6, 0.5, 0.1)
# get_isochrones(grid_logage = grid_logage, grid_mh = grid_mh)

#### 2. Sample mockstar(IMF, $\rm{f}_{b}$)

- define different IMF `fun_IMF(m_x,label)` and `pdf_IMF(m_x,m_min,m_max,label)`

- ramdon sampling according to defined IMF `random_IMF(m_n, m_min, m_max,label)`

In [8]:
# 初始质量函数
# generate the mass catalog follow the selected IMF 
def fun_IMF(m_x,label='Kroupa'):
    '''
    Initial Mass Funcation
    label = ['Salpeter','Kroupa','Chabrier']
    '''
    while label == 'Salpeter':
        if m_x < 0.1:
            return 0
        elif m_x < 100:
            return m_x**(-2.35)
        else : 
            return 0
    
    while label == 'Kroupa':
        if m_x < 0.08 : 
            return 0
        elif m_x < 0.5 : 
            return 2*m_x**(-1.3)
        elif m_x < 150:
            return m_x**(-2.3)
        else :
            return 0
        
        
    while label == 'Chabrier':
        if m_x < 0.07 : 
            return 0
        elif m_x < 1.0 : 
            return m_x**(-1.55)
        elif m_x < 100 : 
            return m_x**(-2.7)
        else : 
            return 0
       
def pdf_IMF(m_x,m_min,m_max,label='Kroupa'):
    '''
    返回IMF的pdf 
    '''
    if m_x < m_min or m_x > m_max:
        return 0
    else : 
        return fun_IMF(m_x,label)/integrate.quad(lambda x : fun_IMF(x,label), m_min, m_max)[0]

def random_IMF(m_n, m_min, m_max,label='Kroupa'):
    '''
    按照 初始质量函数 随机生成 m_n 个范围在 (m_min,m_max) 之间的质量
    '''
    m_result = []
    c = pdf_IMF(m_min, m_min, m_max, label)
    for i in range(m_n):
        m_flag = 0 # 标志，判断生成的 mass 是否已经到达设定的质量上限 m_max
        while m_flag == 0:
            m_x = random.uniform(m_min, m_max) # 从(m_min, m_max)的均匀分布中随机抽取一个数
            m_y = random.uniform(0,1) 
            if m_y < pdf_IMF(m_x, m_min, m_max, label)/c: 
                m_result.append(m_x)
                m_flag = 1
    return m_result

def numericalSort(value):
    '''
    对包含数字的字符串 value 进行自然排序, re 模块实现正则表达式匹配
    自然排序: 与传统的字典序排序(lexical sorting)不同, 在自然排序中数字部分被视为一个整体, 而不是单独的字符
    '''
    numbers = re.compile(r'(\d+)') # 定义一个正则表达式对象 numbers，用于匹配输入字符串中的数字部分
    parts = numbers.split(value) # 使用 numbers.split(value) 将输入字符串分割成若干个部分，其中数字部分和非数字部分交替出现
    parts[1::2] = map(int, parts[1::2]) # 使用 map 函数将数字部分转换成整数，将转换后的数字部分替换原来的字符串部分
    return parts # 返回分割后的结果，其中数字部分已经转换成了整数

def ferr_mag(x,factor=0.1):
    ''' 
    用以计算流量误差
    '''
    return x*factor

In [9]:
class generate_mockCMD:
    '''
    generate mock stars (to draw CMD) from the templated isochrone file ../../data/isochrones/
    self.n_jobs
    self.dist_mod
    self.bands
    self.labelIMF
    self.fbin
    self.complimit
    self.iso_list : 等龄线
    self.n_stars
    '''
    def __init__(self,dist_mod=15,n_jobs=6,labelIMF='Kroupa',fbin=0.35,n_stars=1000,
                 bands = ['G_BPmag','G_RPmag','Gmag'],complimit=0.2):
        '''
        n_jobs = 4
        dist_mod : distance module = m - M, distance in unit 'Mpc'
        bands : list, ['G_BPmag','G_RPmag','Gmag']
        labelIMF : Initial Mass Function, string, ['Kroupa', 'Salpeter', 'Chabrier']
        fbin : binary fraction, float
        '''
        ''' Qs:
        Why dist_mod = 15 ? 为什么DM是一个值?不同的星团应该有不同的DM。
        What does complimit mean ?  
        '''
        self.n_jobs = n_jobs
        self.dist_mod = dist_mod
        self.bands = bands
        self.labelIMF = labelIMF
        self.fbin = fbin
        self.n_stars = n_stars
        self.complimit = complimit
        
    def get_iso(self):
        '''
        get isochrone list from /data/isochrones/
        '''
        cols = ['logAge','MH','Mini','label']
        cols.extend(self.bands)
        '''
        isoc_file = sorted(glob.glob(config['DEFAULT']['PATH_DATA']+'iso/'+config['DEFAULT']['PHOT_SYS']+'/*.csv'),key = numericalSort)
        glob 模块用于查找匹配特定模式的文件路径名
        glob.glob() 接收一个字符串参数, 该参数可以是包含通配符的文件路径模式, 返回符合该模式的所有文件路径名的列表。
        sorted() 对可迭代对象进行排序，并返回一个新的已排序的列表。用 key 参数来指定排序时使用的关键字函数
        '''   
        isoc_file = sorted(glob.glob('../../data/isochrones/*.csv'),key = numericalSort)
        isoc_list = []
        for each in isoc_file:            
            temiso = pd.read_csv('%s'%(each))
            temiso = temiso[cols]
            isoc_list.append(temiso)
        self.isoc_list = isoc_list
    
    def mock_CMD(self,temiso):
        '''
        temiso : 单条等龄线
        mag_limit : maximum magnitude at g band       
        '''
        tem = temiso
        tem = tem[(tem['label']>0)&(tem['label']<=7)] # MS ~ EAGB
        logAge = tem.iloc[0]['logAge']
        MH = tem.iloc[0]['MH']
        tem['Gmag_obs'] = tem['Gmag'] + self.dist_mod
        
        '''
        config['OBJECT']['YMIN'] 与 config['OBJECT']['YMAX'] 中的参数代表什么？
        观测数据的最大最小值？
        fband_mass = spl.interp1d(tem['F814Wmag_obs'],tem['Mini'])
        if float(config['OBJECT']['YMIN']) >  tem['F814Wmag_obs'].min():
            massmax = fband_mass(config['OBJECT']['YMIN'])
        if float(config['OBJECT']['YMIN']) <= tem['F814Wmag_obs'].min():
            massmax = tem.iloc[-1]['Mini']
        if float(config['OBJECT']['YMAX']) >  tem['F814Wmag_obs'].max():
            massmin = fband_mass(tem['F814Wmag_obs'].max())
        if float(config['OBJECT']['YMAX']) <= tem['F814Wmag_obs'].max():
            massmin = fband_mass(config['OBJECT']['YMAX'])
        '''
        massmin = min(tem['Mini'])
        massmax = max(tem['Mini'])
        # 质量
        mass = random_IMF(self.n_stars,massmin,massmax,label=self.labelIMF) # 生成初始质量 primass
        cat = pd.DataFrame(np.zeros((self.n_stars,1)),columns=['primass']) # 创建星表
        cat['primass'] = mass
        cat['age'] = logAge
        cat['mh'] = MH
        # 双星
        secindex = random.sample(list(cat.index),k=int(len(cat)*self.fbin)) # 按双星比 fbin 随机抽取为双星系统的样本
        cat['secmass'] = np.zeros(len(cat))
        bintem = temiso[temiso['label']==1] # MS 主序
        secmass = random_IMF(int(len(cat)*self.fbin),bintem['Mini'].min(),bintem['Mini'].max(),label=self.labelIMF) # 生成双星中伴星的初始质量
        cat['secmass'][secindex] = secmass # 双星系统中伴星的质量
        # 分别对单星和双星系统赋予真实的星等流量 band_true
        for band in self.bands: 
            cat['%s_pri'%(band)] = np.zeros(len(cat))
            cat['%s_sec'%(band)] = np.zeros(len(cat))
            
            if tem.iloc[-1]['label'] <= 3: # MS,SGB,RGB, 如果等时线样本 tem 的阶段到RGB为止
                iso1 = tem
                fmass_band1 = spl.interp1d(iso1['Mini'],iso1['%s'%(band)]) # 拟合流量-质量关系函数
                for h in range(len(cat)):
                    cat['%s_pri'%(band)][h] = fmass_band1(cat['primass'][h]) # 给 cat 表中对应的 primass 赋流量值
                # 对单星，真实流量为其本身的流量
                cat['%s_true'%(band)] = cat['%s_pri'%(band)]
                # 对双星系统
                for h in secindex: 
                    cat['%s_sec'%(band)][h] = fmass_band1(cat['secmass'][h]) # 按照伴星的质量 secmass 赋伴星的流量值
                    cat['%s_true'%(band)][h] = -2.5*np.log10(pow(10,-0.4*cat['%s_pri'%(band)][h])+pow(10,-0.4*cat['%s_sec'%(band)][h]))
            
            if tem.iloc[-1]['label']>3: # CHEB, CHEB_b, CHEB_r, EAGB, 如果等时线样本 tem 存在演化到 RGB 后的阶段 
                iso1 = tem[tem['label']<=3]#MS-RGB
                fmass_band1 = spl.interp1d(iso1['Mini'],iso1['%s'%(band)],fill_value='extrapolate')       
                
                iso2 = tem[tem['label']>=4]#CHEB-EAGB,core He-burning, not include TP-AGB
                fmass_band2 = spl.interp1d(iso2['Mini'],iso2['%s'%(band)])
                mass_cut = iso2['Mini'].min() # CHEB 开始时的质量
                
                for h in range(len(cat)):
                    if cat['primass'][h] < mass_cut: # 若初始质量 primass < EAGB 开始时的质量(处于 MS-RGB 阶段)
                        cat['%s_pri'%(band)][h] = fmass_band1(cat['primass'][h]) # 按照 MS-RGB 阶段的流量-质量关系赋值
                    else : # 处于 CHEB-EAGB 阶段
                        cat['%s_pri'%(band)][h] = fmass_band2(cat['primass'][h])
                # 对单星，真实流量为其本身的流量
                cat['%s_true'%(band)] = cat['%s_pri'%(band)]
                # 对双星系统
                for h in secindex:
                    if cat['secmass'][h] < mass_cut:
                        cat['%s_sec'%(band)][h] = fmass_band1(cat['secmass'][h])
                    else :
                        cat['%s_sec'%(band)][h] = fmass_band2(cat['secmass'][h])
                    cat['%s_true'%(band)][h] = -2.5*np.log10(pow(10,-0.4*cat['%s_pri'%(band)][h])+pow(10,-0.4*cat['%s_sec'%(band)][h]))           
        # 流量误差Error(涉及测光)
        for band in self.bands:
            cat['%s'%(band)] = cat['%s_true'%(band)] + self.dist_mod # 1、改正距离模数
            # uncertainties 无信息(无UNCER_FIT.csv文件)，定义 ferr_mag 函数代替 (感觉有问题？)
            # ferr_mag = spl.interp1d(uncer['mag'],uncer['%s_err'%(band)],fill_value='extrapolate')
            cat['%s_obs'%(band)] = np.zeros(len(cat))
    
            for i in range(len(cat)):
                temmag = cat['%s'%(band)][i] # 改正距离模数后的流量
                temerr = ferr_mag(temmag) # 再现 color spread, 流量的“人造误差”
                cat['%s_obs'%(band)][i] = random.gauss(temmag,temerr) # 2、观测流量从 N(temmag,temerr) 随机抽样获得
            cat['%s_err'%(band)] = cat['%s_obs'%(band)]-cat['%s'%(band)] # 流量的实际误差
        
        # 保留 ｜流量误差｜< 0.5 的数据
        for band in self.bands:
            cat = cat[(cat['%s_err'%(band)]<0.5)&(cat['%s_err'%(band)]>-0.5)].reset_index(drop=True)
     
        # apply completeness map to the ssp with uncertainties
        ''' 
        # xbins, ybins 是啥？
        xbins = np.arange(float(config['OBJECT']['XMIN']),float(config['OBJECT']['XMAX']),float(config['OBJECT']['XBINSIZE']))
        ybins = np.arange(float(config['OBJECT']['YMIN']),float(config['OBJECT']['YMAX']),float(config['OBJECT']['YBINSIZE']))
        grid_x,grid_y = np.meshgrid(xbins,ybins)
        grid_z = np.zeros((len(ybins),len(xbins))) 
        '''
        cat['color_obs'] = cat['%s_obs'%(self.bands[0])] - cat['%s_obs'%(self.bands[1])] # BP-RP
        cat['mag_obs'] =   cat['%s_obs'%(self.bands[2])] # G
        temcat = cat.dropna()
        '''
        temcat = temcat[(temcat['color_obs']>=xbins[0])&(temcat['color_obs']<=xbins[-1])&(temcat['mag_obs']>=ybins[0])&(temcat['mag_obs']<=ybins[-1])]

        drop_index = []
        if len(temcat)>0:
            for i in range(len(ybins)-1):
                temcaty = temcat[(temcat['mag_obs']>=ybins[i])&(temcat['mag_obs']<ybins[i+1])]
                for j in range(len(xbins)-1):
                    temcatxy = temcaty[(temcaty['color_obs']>=xbins[j])&(temcaty['color_obs']<xbins[j+1])]
                    if cmap[i,j] >= float(config['OBJECT']['COMPLIMIT']):
                        temdrop_index = random.choices(temcatxy.index,k=int(len(temcatxy)*(1-cmap[i,j])))
                        drop_index.extend(temdrop_index)
                    if cmap[i,j] <  float(config['OBJECT']['COMPLIMIT']):
                        drop_index.extend(temcatxy.index)
                    grid_z[i,j] = len(temcatxy)
        savecat = temcat.drop(drop_index)
        savecat.to_csv('../../data/catalogues/cat_age_%s_mh_%s.csv'%(logAge,MH),index=False)
        '''
        temcat.to_csv('../../data/catalogues/cat_age_%s_mh_%s.csv'%(logAge,MH),index=False)

        with open('../../data/ncat/ncat_age_%s_mh_%s.txt'%(logAge,MH),'a') as file:
            file.write('%-20s %-20s \n'%('N_ini' , len(cat)))
            file.write('%-20s %-20s \n'%('N_view', len(temcat)))
            # file.write('%-20s %-20s \n'%('N_comp', len(savecat)))
    
    def generate_parallel(self):
        ''' 
        Parallel 类将一个可迭代对象中的任务并行化执行
        Parallel(n_jobs=self.n_jobs)创建了一个Parallel对象，其中n_jobs参数表示并行执行的作业数
        (delayed(self.mock_CMD)(temiso=_iso) for _iso in (self.isoc_list))是一个生成器表达式，用于生成要执行的任务。每个任务都是调用self.mock_CMD()方法并传递一个参数temiso=_iso
        
        '''
        Parallel(n_jobs=self.n_jobs)(delayed(self.mock_CMD)(temiso=_iso) for _iso in (self.isoc_list))
        

In [10]:
# test using Melotte_22, DM=5.55
f = generate_mockCMD(dist_mod = 5.55,
                     n_jobs = 6,
                     fbin = 0.35,
                     n_stars = 1000,
                     bands = ['G_BPmag','G_RPmag','Gmag'],
                     complimit=0.2)
f.get_iso()
f.generate_parallel()

/tmp/ipykernel_2727287/3106513350.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_2727287/3106513350.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_2727287/3106513350.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykerne